In [1]:
from pytdx.hq import TdxHq_API
from pytdx.params import TDXParams
from datetime import datetime, timedelta
import pandas as pd
import polars as pl
import os
from tqdm import tqdm

In [2]:
symbol = '002812'

df = pl.read_parquet(f'data/{symbol}.parquet')
df.head()

open,close,high,low,vol,amount,datetime
f64,f64,f64,f64,f64,f64,str
50.5,50.95,51.45,49.55,7.902809e6,4.00563104e8,"""2020-01-02 15:00"""
50.84,53.88,53.88,50.11,9.706831e6,5.11194048e8,"""2020-01-03 15:00"""
55.12,56.53,58.88,55.12,1.5652109e7,8.88578688e8,"""2020-01-06 15:00"""
57.49,56.83,57.79,55.99,8.364574e6,4.74544064e8,"""2020-01-07 15:00"""
56.96,55.41,58.15,55.41,1.0989446e7,6.20674368e8,"""2020-01-08 15:00"""


In [3]:
dts = []
for dt_str in df['datetime']:
    dts.append(datetime.strptime(dt_str.split(' ')[0], '%Y-%m-%d'))

df['datetime'] = dts
df.head()

open,close,high,low,vol,amount,datetime
f64,f64,f64,f64,f64,f64,datetime
50.5,50.95,51.45,49.55,7.902809e6,4.00563104e8,2020-01-02 00:00:00
50.84,53.88,53.88,50.11,9.706831e6,5.11194048e8,2020-01-03 00:00:00
55.12,56.53,58.88,55.12,1.5652109e7,8.88578688e8,2020-01-06 00:00:00
57.49,56.83,57.79,55.99,8.364574e6,4.74544064e8,2020-01-07 00:00:00
56.96,55.41,58.15,55.41,1.0989446e7,6.20674368e8,2020-01-08 00:00:00


In [4]:
ls = df['low']
df['pivot_low'] = ((ls <= ls.shift(1)).fill_null(False) & (ls <= ls.shift(-1)).fill_null(False))
df['pivot_low_price'] = df['pivot_low'] * df['low']
df['pivot_low_price'] = df['pivot_low_price'].apply(lambda x: x if x > 0 else None)

hs = df['high']
df['pivot_high'] = ((hs >= hs.shift(1)).fill_null(False) & (hs >= hs.shift(-1)).fill_null(False))
df['pivot_high_price'] = df['pivot_high'] * df['high']
df['pivot_high_price'] = df['pivot_high_price'].apply(lambda x: x if x > 0 else None)

In [5]:
df['freq'] = ['1d'] * len(df)
df['symbol'] = [symbol] * len(df)

In [6]:
df.to_parquet(f'../../analyzer.view.server/dump/cn_pivot/compute/{symbol}.1d.parquet')

- Trend Logic
    - 向前找 valid pivot
        - Lower High
        - Higher Low
    - 持续多久
    - 两个valid pivot间间隔多少
        - period
        - pivot
        - price diff
        - revert trend
    - Trend Break
    - 当前 Bar/价
        - Potentially making High/Low
        - Potentially continuing trend
        - Potentially breaking trend
    - 如果趋势延续,预测下一个 pivot 的位置
    - Multi-periods
- Feature
    - 趋势持续的周期
    - 趋势刚开始 / 已持续长时间
    - Reversal
    - Chanel
- Strategy
    - 趋势反转
    - 趋势启动 -> Chanel
- Visual